經過第一輪嘅標註同處理之後，我哋要做下面步驟：

1. 將 `round_1_annotated/output/error_words.csv`同`round_1_annotated/re_output/error_words.csv`入面嘅詞條人工修正然後擺落 `fixed.csv` 度
1. 將


放到上游詞表嘅文件係 `word.csv`

1. 將所有 `error_words.csv` 嘅詞表修正，加入總表  
1. 將所有 `jyutping_useful.csv` 嘅詞加入總表
1. 將所有 `no_jyutping_useful.csv` 嘅詞
  1. 用代碼加入粵拼
  1. （揾人做）檢查粵拼
  1. 加入總表
1. （揾人做）將所有 `jyutping_junk.csv` 嘅詞，人手篩選有用嘅加入總表

`no_jyutping_junk.csv`入面嘅詞條人工檢查一部分，確定冇漏之後全部丟棄。跟住將總表入面嘅所有詞彙進行下面嘅處理工作：

1. （揾人做）整理出所有同音詞，然後討論規範寫法，刪除唔標準嘅寫法
1. 將所有詞條轉成 OpenCC 字形
1. 去重排序

跟住就**發佈初版**，然後開始下一輪嘅人工處理：

1. 將詞表分成幾個類別
1. 